In [ ]:
pip install Keras==2.3.1

In [1]:
import pandas as pd
import numpy as np
import pickle
np.random.seed(1212)
import keras
from keras.models import Model
from keras.layers import *
from keras import optimizers
from keras.layers import Input, Dense
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_data_format('channels_first')

Using TensorFlow backend.


In [2]:
df_train=pd.read_csv('train_final.csv',index_col=False)
labels=df_train[['784']]

In [3]:
df_train.drop(df_train.columns[[784]],axis=1,inplace=True)
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,255,255,255,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
labels=np.array(labels)

In [5]:
from keras.utils.np_utils import to_categorical
cat=to_categorical(labels,num_classes=13)

In [6]:
print(cat[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [7]:
df_train.shape

(125269, 784)

In [8]:
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,255,255,255,255,255,255,255,255,255,255,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
l=[]
for i in range(125269):
    l.append(np.array(df_train[i:i+1]).reshape(1, 28 , 28))

In [10]:
np.random.seed(7)

In [11]:
from tensorflow.keras.models import load_model

In [12]:
import tensorflow as tf
import keras.backend.tensorflow_backend as tfback

print("tf.__version__ is", tf.__version__)
print("tf.keras.__version__ is:", tf.keras.__version__)

def _get_available_gpus():
    """Get a list of available gpu devices (formatted as strings).

    # Returns
        A list of available GPU devices.
    """
    #global _LOCAL_DEVICES
    if tfback._LOCAL_DEVICES is None:
        devices = tf.config.list_logical_devices()
        tfback._LOCAL_DEVICES = [x.name for x in devices]
    return [x for x in tfback._LOCAL_DEVICES if 'device:gpu' in x.lower()]

tfback._get_available_gpus = _get_available_gpus

tf.__version__ is 2.2.0
tf.keras.__version__ is: 2.3.0-tf


In [13]:
model = Sequential()
model.add(Conv2D(30, (5, 5), input_shape=(1, 28 , 28), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(15, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(13, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
from keras.models import model_from_json

In [15]:
model.fit(np.array(l), cat, epochs=10, batch_size=200,shuffle=True,verbose=1)

Epoch 1/10
125269/125269 [==============================] - 85s 681us/step - loss: 0.6237 - accuracy: 0.8481
Epoch 2/10
125269/125269 [==============================] - 82s 657us/step - loss: 0.1404 - accuracy: 0.9588
Epoch 3/10
125269/125269 [==============================] - 85s 679us/step - loss: 0.0885 - accuracy: 0.9737
Epoch 4/10
125269/125269 [==============================] - 84s 671us/step - loss: 0.0682 - accuracy: 0.9794
Epoch 5/10
125269/125269 [==============================] - 84s 671us/step - loss: 0.0533 - accuracy: 0.9840
Epoch 6/10
125269/125269 [==============================] - 86s 683us/step - loss: 0.0472 - accuracy: 0.9855
Epoch 7/10
125269/125269 [==============================] - 85s 678us/step - loss: 0.0396 - accuracy: 0.9876
Epoch 8/10
125269/125269 [==============================] - 86s 683us/step - loss: 0.0354 - accuracy: 0.9892
Epoch 9/10
125269/125269 [==============================] - 85s 676us/step - loss: 0.0324 - accuracy: 0.9901
Epoch 10/10
125269/

In [16]:
model_json = model.to_json()
with open("model_final.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_final.h5")